# Profiling BatchFlow code

A profile is a set of statistics that describes how often and for how long various parts of the program executed.

This notebooks shows how to profile various parts of BatchFlow: namely, pipelines and models.

In [1]:
import sys

sys.path.append("../../..")
from batchflow import B, V, W
from batchflow.opensets import MNIST
from batchflow.models.torch import ResNet18

In [2]:
dataset = MNIST()

To collect information about model training times (both on CPU and GPU), one must set `profile` option in the model configuration to `True`:

In [3]:
model_config = {
    'inputs/labels/classes': 10,
    'loss': 'ce',
    'profile': True,
    'device': 'cpu',
    'profile': False,
    'amp': False
}

In [4]:
pipeline = (dataset.train.p
                .init_variable('loss_history', [])
                .to_array(channels='first', dtype='float32')
                .multiply(multiplier=1/255., preserve_type=False)
                .init_model('resnet', ResNet18, 'dynamic',
                            config=model_config)
                .train_model('resnet',
                             B.images, B.labels,
                             fetches='loss',
                             save_to=V('loss_history', mode='a'))
                )

To gather statistics about how long each action takes, we must set `profile` to `True` inside `run` call:

In [5]:
BATCH_SIZE = 64
N_ITERS = 50

pipeline.run(BATCH_SIZE, n_iters=N_ITERS, bar=True, profile='d')

  0%|                                                    | 0/50 [00:00<?, ?it/s]/opt/conda/lib/python3.6/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|███████████████████████████████████████████| 50/50 [00:11<00:00,  4.27it/s]


# Pipeline profiling

First of all, there is an `elapsed_time` attribute inside every instance of `Pipeline`: it stores total time of running the pipeline (even if it was used multiple times):

In [6]:
pipeline.elapsed_time

AttributeError: elapsed_time not found in class Pipeline

Note that `elapsed_time` attribute is created whether or not we set `profile` to `True`.

After running with `profile=True`, pipeline has attribute `profile_info`: this `DataFrame` holds collected information:

In [7]:
pipeline.profile_info.head()

iter  \
action      id                                                         
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...     1   
            <built-in method sys.getrecursionlimit>::/opt/c...     1   
            <built-in method time.time>::../../../batchflow...     1   
            <method 'match' of '_sre.SRE_Pattern' objects>:...     1   
            <method 'match' of '_sre.SRE_Pattern' objects>:...     1   

                                                                total_time  \
action      id                                                               
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...    0.009355   
            <built-in method sys.getrecursionlimit>::/opt/c...    0.009355   
            <built-in method time.time>::../../../batchflow...    0.009355   
            <method 'match' of '_sre.SRE_Pattern' objects>:...    0.009355   
            <method 'match' of '_sre.SRE_Pattern' objects>:...    0.009355   

                                                                pipeline_time  \
action      id                                                                  
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...       0.009332   
            <built-in method sys.getrecursionlimit>::/opt/c...       0.009332   
            <built-in method time.time>::../../../batchflow...       0.009332   
            <method 'match' of '_sre.SRE_Pattern' objects>:...       0.009332   
            <method 'match' of '_sre.SRE_Pattern' objects>:...       0.009332   

                                                                ncalls  \
action      id                                                           
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...      64   
            <built-in method sys.getrecursionlimit>::/opt/c...       1   
            <built-in method time.time>::../../../batchflow...       2   
            <method 'match' of '_sre.SRE_Pattern' objects>:...       1   
            <method 'match' of '_sre.SRE_Pattern' objects>:...      66   

                                                                 tottime  \
action      id                                                             
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...  0.000009   
            <built-in method sys.getrecursionlimit>::/opt/c...  0.000001   
            <built-in method time.time>::../../../batchflow...  0.000001   
            <method 'match' of '_sre.SRE_Pattern' objects>:...  0.000004   
            <method 'match' of '_sre.SRE_Pattern' objects>:...  0.000129   

                                                                 cumtime  \
action      id                                                             
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...  0.000009   
            <built-in method sys.getrecursionlimit>::/opt/c...  0.000001   
            <built-in method time.time>::../../../batchflow...  0.000001   
            <method 'match' of '_sre.SRE_Pattern' objects>:...  0.000004   
            <method 'match' of '_sre.SRE_Pattern' objects>:...  0.000129   

                                                                  start_time  \
action      id                                                                 
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...  1.626968e+09   
            <built-in method sys.getrecursionlimit>::/opt/c...  1.626968e+09   
            <built-in method time.time>::../../../batchflow...  1.626968e+09   
            <method 'match' of '_sre.SRE_Pattern' objects>:...  1.626968e+09   
            <method 'match' of '_sre.SRE_Pattern' objects>:...  1.626968e+09   

                                                                       batch_id  \
action      id                                                                    
to_array #0 <method 'join' of 'bytes' objects>::/opt/conda/...  139894849693624   
            <built-in method sys.getrecursionlimi

Note that there is a detailed information about exact methods that are called inside each of the actions. That is a lot of data which can give us precise understanding of parts of the code, that are our bottlenecks.

Columns of the `profile_info`:
- `action`, `iter`, `batch_id` and `start_time` are pretty self-explainable
- `id` allows to identify exact method with great details: it is a concatenation of `method_name`, `file_name`, `line_number` and `callee`
- `total_time` is a time taken by an action
- `pipeline_time` is `total_time` plus time of processing the profiling table at each iteration
- `tottime` is a time taken by a method inside action
- `cumtime` is a time take by a method and all of the methods that are called inside this method


More often than not, though, we don't need such granularity. Pipeline method `show_profile_info` makes some handy aggregations:

**Note:** by default, results are sorted on `total_time` or `tottime`, depending on level of details.

In [8]:
# timings for each action
pipeline.show_profile_info(per_iter=False, detailed=False)

total_time                     pipeline_time            \
                      sum      mean       max           sum      mean   
action                                                                  
train_model #2   8.140517  0.162810  0.709740      8.136595  0.162732   
to_array #0      0.463756  0.009275  0.016101      0.462688  0.009254   
multiply #1      0.337302  0.006746  0.009966      0.336401  0.006728   

                          
                     max  
action                    
train_model #2  0.707148  
to_array #0     0.016080  
multiply #1     0.009932

In [9]:
# for each action show 2 of the slowest methods, based on maximum `ncalls`
pipeline.show_profile_info(per_iter=False, detailed=True, sortby=('ncalls', 'max'), limit=2)

ncalls  \
                                                                     sum   
action         id                                                          
multiply #1    <built-in method builtins.isinstance>::../../.....   9600   
               <built-in method numpy.core._multiarray_umath.i...   6500   
to_array #0    <built-in method builtins.isinstance>::../../.....   9600   
               <built-in method numpy.core._multiarray_umath.i...   6500   
train_model #2 <method 'get' of 'dict' objects>::/opt/conda/li...  45216   
               <built-in method builtins.isinstance>::/opt/con...  31335   

                                                                           \
                                                                     mean   
action         id                                                           
multiply #1    <built-in method builtins.isinstance>::../../.....  192.00   
               <built-in method numpy.core._multiarray_umath.i...  130.00   
to_array #0    <built-in method builtins.isinstance>::../../.....  192.00   
               <built-in method numpy.core._multiarray_umath.i...  130.00   
train_model #2 <method 'get' of 'dict' objects>::/opt/conda/li...  904.32   
               <built-in method builtins.isinstance>::/opt/con...  626.70   

                                                                          \
                                                                     max   
action         id                                                          
multiply #1    <built-in method builtins.isinstance>::../../.....    192   
               <built-in method numpy.core._multiarray_umath.i...    130   
to_array #0    <built-in method builtins.isinstance>::../../.....    192   
               <built-in method numpy.core._multiarray_umath.i...    130   
train_model #2 <method 'get' of 'dict' objects>::/opt/conda/li...  10083   
               <built-in method builtins.isinstance>::/opt/con...   6933   

                                                                    tottime  \
                                                                        sum   
action         id                                                             
multiply #1    <built-in method builtins.isinstance>::../../.....  0.001557   
               <built-in method numpy.core._multiarray_umath.i...  0.015555   
to_array #0    <built-in method builtins.isinstance>::../../.....  0.001970   
               <built-in method numpy.core._multiarray_umath.i...  0.020430   
train_model #2 <method 'get' of 'dict' objects>::/opt/conda/li...  0.009140   
               <built-in method builtins.isinstance>::/opt/con...  0.006680   

                                                                             \
                                                                       mean   
action         id                                                             
multiply #1    <built-in method builtins.isinstance>::../../.....  0.000031   
               <built-in method numpy.core._multiarray_umath.i...  0.000311   
to_array #0    <built-in method builtins.isinstance>::../../.....  0.000039   
               <built-in method numpy.core._multiarray_umath.i...  0.000409   
train_model #2 <method 'get' of 'dict' objects>::/opt/conda/li...  0.000183   
               <built-in method builtins.isinstance>::/opt/con...  0.000134   

                                                                             \
                                                                        max   
action         id                                                             
multiply #1    <built-in method builtins.isinstance>::../../.....  0.000053   
               <built-in method numpy.core._multiarray_umath.i...  0.000448   
to_array #0    <built-in method builtins.isinstance>::../../.....  0.000092   
               <built-in method numpy.core._multiarray_umath.i...  0.000880   
train_model #2 <method 'g

In [10]:
# timings for each action for each iter
pipeline.show_profile_info(per_iter=True, detailed=False,)

total_time  pipeline_time      batch_id
iter action                                                 
1    train_model #2    0.709740       0.707148  1.398948e+14
     to_array #0       0.009355       0.009332  1.398948e+14
     multiply #1       0.007039       0.007026  1.398948e+14
2    train_model #2    0.139050       0.139026  1.398948e+14
     to_array #0       0.007055       0.007037  1.398948e+14
...                         ...            ...           ...
49   to_array #0       0.008777       0.008757  1.398948e+14
     multiply #1       0.006446       0.006428  1.398948e+14
50   train_model #2    0.147496       0.147471  1.398948e+14
     to_array #0       0.009047       0.009030  1.398948e+14
     multiply #1       0.006508       0.006494  1.398948e+14

[150 rows x 3 columns]

In [11]:
# for each iter each action show 3 of the slowest methods, based on maximum `ncalls`
pipeline.show_profile_info(per_iter=True, detailed=True, sortby='tottime', limit=3)

ncalls  \
iter action         id                                                           
1    multiply #1    <method 'spawn' of 'numpy.random.bit_generator....      64   
                    inner::../../../batchflow/batchflow/utils_rando...      64   
                    multiply::../../../batchflow/batchflow/batch.py...      64   
     to_array #0    <method 'spawn' of 'numpy.random.bit_generator....      64   
                    inner::../../../batchflow/batchflow/utils_rando...      64   
...                                                                        ...   
50   to_array #0    inner::../../../batchflow/batchflow/utils_rando...      64   
                    <method 'astype' of 'numpy.ndarray' objects>::....      64   
     train_model #2 <method 'run_backward' of 'torch._C._EngineBase...       1   
                    adam::/opt/conda/lib/python3.6/site-packages/to...       1   
                    <built-in method conv2d>::/opt/conda/lib/python...      20   

                                                                         tottime  \
iter action         id                                                             
1    multiply #1    <method 'spawn' of 'numpy.random.bit_generator....  0.001124   
                    inner::../../../batchflow/batchflow/utils_rando...  0.000866   
                    multiply::../../../batchflow/batchflow/batch.py...  0.000419   
     to_array #0    <method 'spawn' of 'numpy.random.bit_generator....  0.001297   
                    inner::../../../batchflow/batchflow/utils_rando...  0.000951   
...                                                                          ...   
50   to_array #0    inner::../../../batchflow/batchflow/utils_rando...  0.000962   
                    <method 'astype' of 'numpy.ndarray' objects>::....  0.000642   
     train_model #2 <method 'run_backward' of 'torch._C._EngineBase...  0.052696   
                    adam::/opt/conda/lib/python3.6/site-packages/to...  0.024636   
                    <built-in method conv2d>::/opt/conda/lib/python...  0.015681   

                                                                         cumtime  
iter action         id                                                            
1    multiply #1    <method 'spawn' of 'numpy.random.bit_generator....  0.001447  
                    inner::../../../batchflow/batchflow/utils_rando...  0.001536  
                    multiply::../../../batchflow/batchflow/batch.py...  0.000608  
     to_array #0    <method 'spawn' of 'numpy.random.bit_generator....  0.001847  
                    inner::../../../batchflow/batchflow/utils_rando...  0.001727  
...                                                                          ...  
50   to_array #0    inner::../../../batchflow/batchflow/utils_rando...  0.001654  
                    <method 'astype' of 'numpy.ndarray' objects>::....  0.000642  
     train_model #2 <method 'run_backward' of 'torch._C._EngineBase...  0.052696  
                    adam::/opt/conda/lib/python3.6/site-packages/to...  0.037825  
                    <built-in method conv2d>::/opt/conda/lib/python...  0.015681  

[450 rows x 3 columns]

# Model profiling

In [ ]:
model = pipeline.m('resnet')

There is an `info` property that, unsurprisingly, shows a lot of interesting details regarding model itself or the training process:

In [ ]:
model.info

As with pipeline, there is a `profile_info` attribute, as well as `show_profile_info` method. Depending on type of the used device (`CPU` or `GPU`)

In [ ]:
# one row for every operation inside model; limit at 5 rows
model.show_profile_info(per_iter=False, limit=5)

In [ ]:
# for each iteration show 3 of the slowest operations
model.show_profile_info(per_iter=True, limit=3)